In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re
import requests
import urllib
from selenium.webdriver.common.by import By
from lxml import etree
import random

In [2]:
df = pd.read_csv('HPI_master.csv')

In [5]:
df.columns

Index(['hpi_type', 'hpi_flavor', 'frequency', 'level', 'place_name',
       'place_id', 'yr', 'period', 'index_nsa', 'index_sa'],
      dtype='object')

In [3]:
pd.read_csv('HPI_AT_metro.csv')

,"Abilene, TX",10180,1975,1,-,-.1
0,"Abilene, TX",10180,1975,2,-,-
1,"Abilene, TX",10180,1975,3,-,-
2,"Abilene, TX",10180,1975,4,-,-
3,"Abilene, TX",10180,1976,1,-,-
4,"Abilene, TX",10180,1976,2,-,-
...,...,...,...,...,...,...
77562,"Yuma, AZ",49740,2021,4,271.47,( 3.60)
77563,"Yuma, AZ",49740,2022,1,285.43,( 3.89)
77564,"Yuma, AZ",49740,2022,2,302.93,( 4.43)
77565,"Yuma, AZ",49740,2022,3,312.46,( 4.98)


In [4]:
df2 = pd.read_csv('RDC_Inventory_Hotness_Metrics_Metro_History.csv')

In [5]:
df2['cbsa_title'].astype(str)

0                                Rochester, NY
1                        Manchester-Nashua, NH
2        Hartford-East Hartford-Middletown, CT
3                              Springfield, MA
4                                 Columbus, OH
                         ...                  
20096                          Columbus, GA-AL
20097                               Dothan, AL
20098                  Naples-Marco Island, FL
20099                      Houma-Thibodaux, LA
20100                                      nan
Name: cbsa_title, Length: 20101, dtype: object

In [6]:
state = []
cbsa = []

for i in df2['cbsa_title'].astype(str):
    cbsa_state = i.split(",")
    if len(cbsa_state) == 2:
        state.append(cbsa_state[1].strip())
        cbsa.append(cbsa_state[0])
    else:
        continue
    


In [7]:
url ="https://www.opentable.com/columbus-restaurant-listings"

with urllib.request.urlopen(url) as response:
    html = response.read()



soup = BeautifulSoup(html,'lxml')

In [8]:
for i in soup.find_all('div',class_ = 'rj77IvI83cyWF7lgKIEx'):
    print(i.text)

The Keep Kitchen & Liquor Bar
Kona Grill - Columbus
Eddie Merlot's Prime Aged Beef & Seafood - Columbus


In [9]:
soup.find_all('div', class_ = 'rest-row-info')


[]

In [32]:
def parse_html(html):
    """Parse content from various tags from OpenTable restaurants listing"""
    data, item = pd.DataFrame(), {}
    soup = BeautifulSoup(html, 'lxml')
    for i, resto in enumerate(soup.find_all('div', class_='rest-row-info')):
        item['name'] = resto.find('span', class_='rest-row-name-text').text

        booking = resto.find('div', class_='booking')
        item['bookings'] = re.search('\d+', booking.text).group() if booking else 'NA'

        rating = resto.find('div', class_='star-rating-score')
        item['rating'] = float(rating['aria-label'].split()[0]) if rating else 'NA'

        reviews = resto.find('span', class_='underline-hover')
        item['reviews'] = int(re.search('\d+', reviews.text).group()) if reviews else 'NA'

        item['price'] = int(resto.find('div', class_='rest-row-pricing').find('i').text.count('$'))
        item['cuisine'] = resto.find('span', class_='rest-row-meta--cuisine rest-row-meta-text sfx1388addContent').text
        item['location'] = resto.find('span', class_='rest-row-meta--location rest-row-meta-text sfx1388addContent').text
        data[i] = pd.Series(item)
    return data.T


# Start selenium and click through pages until reach end
# store results by iteratively appending to csv file
driver = webdriver.Chrome()
url = "https://www.opentable.com/columbus-restaurant-listings"
driver.get(url)
page = collected = 0
while True:
    sleep(1)
    new_data = parse_html(driver.page_source)
    if new_data.empty:
        break
    if page == 0:
        new_data.to_csv('results.csv', index=False)
    elif page > 0:
        new_data.to_csv('results.csv', index=False, header=None, mode='a')
    page += 1
    collected += len(new_data)
    print(f'Page: {page} | Downloaded: {collected}')
    driver.find_element_by_link_text('Next').click()

driver.close()
#restaurants = pd.read_csv('results.csv')
#print(restaurants)



""


In [34]:
parse_html(html)

""


In [6]:
url = "https://www.opentable.com/columbus-restaurant-listings"

driver = webdriver.Chrome(r"C:\Users\wilke\Downloads\chromedriver_win32\chromedriver.exe")

driver.get(url)

soup = BeautifulSoup(driver.page_source)

all_cards = []

page_count = 0

for i in soup.find('ul', class_ = 'BklkG9D8lgpsm_Z7JPTQ').children:
    page_count += 1

for i in range(page_count):
    scroll = 720
    height = driver.execute_script('return document.body.scrollHeight')
    while scroll < height:
        sleep(3)
        driver.execute_script(f"window.scrollTo(0, {scroll})")
        scroll += 720

    soup = BeautifulSoup(driver.page_source)
    cards = soup.find_all('div', class_ = 'BhIFAK4B5RMKaTMhR78N')

    all_cards.append(cards)

    if i < page_count-1:
        driver.find_element(By.CSS_SELECTOR,"[aria-label = 'Go to the next page']").click()










In [25]:
rest_name_loc = []
for i in all_cards[0]:
    rest_name_loc.append(i.text[:i.text.find('Booked')])


In [33]:
len(rest_name_loc)

47

In [34]:
rest_names = []
for i in rest_name_loc:
    if i.find('4') != -1:
        rest = i[:(i.find('4'))]
        rest_names.append(rest)

In [35]:
len(rest_names)

44

In [18]:
url = "https://www.realtor.com/realestateandhomes-search/Columbus_OH"


from selenium.webdriver.firefox.options import Options

options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

driver = webdriver.Firefox(executable_path=r"C:\Users\wilke\OneDrive\Documents\Selenium\geckodriver.exe", options=options)



driver.get(url)

sleep(random.randint(3,15))

soup = BeautifulSoup(driver.page_source)


dom = etree.HTML(str(soup))

items = []

for i in soup.find('ul',{'data-testid': 'property-list-container'}).children:
    item = i.get('id')
    if item[0] == 'i':
        items.append(item)

for_sale_list = []
beds_list = []
baths_list = []
sqft_list = []
acres_list = []
price_list = []
address_list = []
city_list = []

for page in range(50):
    for i in items:
        for_sale = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[1]/div/span[2]')[0].text
        try:
            beds = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[1]/span[1]')[0].text
        except:
            beds = 'NA'
        try:
            baths = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[2]/span[1]')[0].text
        except:
            baths = 'NA'
        try:
            sqft = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[3]/span[1]')[0].text
        except:
            sqft = 'NA'
        try:
            acres = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[4]/span[1]')[0].text
        except IndexError:
            acres = 'NA'

        price = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[1]/span')[0].text

        try:
            address = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[3]/div[1]')[0].text
        except:
            address = 'NA'
        try:
            city = dom.xpath(f'//*[@id="{i}"]/div/div[2]/div[4]/div[2]/div/div[3]/div[1]/div')[0].text
        except:
            city = 'NA'

        for_sale_list.append(for_sale)
        beds_list.append(beds)
        baths_list.append(baths)
        sqft_list.append(sqft)
        acres_list.append(acres)
        price_list.append(price)
        address_list.append(address)
        city_list.append(city)

    

    driver.find_element(By.CSS_SELECTOR,"[aria-label = 'Go to next page']").click()

    sleep(random.randint(3,15))

    soup = BeautifulSoup(driver.page_source)

    dom = etree.HTML(str(soup))

    items = []

    for i in soup.find('ul',{'data-testid': 'property-list-container'}).children:
        item = i.get('id')
        print(item)
        if item[0] == 'i':
            items.append(item)
    
    
    sleep(random.randint(3,15))







item_3230911342_0
item_417000586288_1
item_4211091534_2
ad_container_TILELIST1
item_4615399679_3
item_417000732919_4
item_3933486156_5
item_3128109097_6
item_4269226404_7
ad_container_TILELIST2
item_4025372598_8
item_3739591457_9
ad_container_INLINELIST1
item_3108518583_10
item_4258843728_11
item_3631121671_12
ad_container_TILELIST3
item_4076736439_13
item_4007496287_14
item_3112468773_15
item_3414595172_16
item_4844116696_17
ad_container_TILELIST4
item_4390117122_18
item_4525130194_19
ad_container_INLINELIST2
item_4275338019_20
item_3120425280_21
item_3780360398_22
ad_container_TILELIST5
item_4776307232_23
item_4201220764_24
item_4483322392_25
item_4521926760_26
item_3739307820_27
ad_container_TILELIST6
item_4095704804_28
item_3991732824_29
ad_container_INLINELIST3
item_3605098671_30
item_3436112112_31
item_4565963731_32
ad_container_TILELIST7
item_3938554324_33
item_4685773737_34
item_3052582434_35
item_9602850908_36
item_3884369613_37
ad_container_TILELIST8
item_4668493712_38
item_4

In [19]:
real_estate_df = pd.DataFrame({
    'Status':for_sale_list,
    'Beds':beds_list,
    'Baths':baths_list,
    'SQFT': sqft_list,
    'Acres':acres_list,
    'Price':price_list,
    'Address': address_list,
    'City':city_list
})

In [22]:
for_sale_homes = real_estate_df[real_estate_df['Status'] == 'For Sale']

In [79]:
#forsale
print(dom.xpath('//*[@id="item_4250458860"]/div/div[2]/div[4]/div[1]/div/span[2]')[0].text)

#beds
print(dom.xpath('//*[@id="item_4250458860"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[1]/span[1]')[0].text)

#baths
print(dom.xpath('//*[@id="item_4193754647"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[2]/span[1]')[0].text)

#sqft
print(dom.xpath('//*[@id="item_4193754647"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[3]/span[1]')[0].text)

#acres
print(dom.xpath('//*[@id="item_9385960324"]/div/div[2]/div[4]/div[2]/div/div[2]/div/ul/li[3]/span[1]')[0].text)

#price
print(dom.xpath('//*[@id="item_9385960324"]/div/div[2]/div[4]/div[2]/div/div[1]/span')[0].text)

#address
print(dom.xpath('//*[@id="property_9385960324"]/div[2]/div[4]/div[2]/div/div[3]/div[1]')[0].text)

#city
print(dom.xpath('//*[@id="property_9385960324"]/div[2]/div[4]/div[2]/div/div[3]/div[1]/div')[0].text)





For Sale
4
2
2,081
0.35
$595,000
6386-88 Wolcott Ct
Dublin


In [25]:
for_sale_homes.drop('Acres', axis =1)

,Status,Beds,Baths,SQFT,Price,Address,City
0,For Sale,3,2,"2,081","$250,000",8234 Bertson Pl,Columbus
1,For Sale,4,2.5,"2,229","$220,000",7448 Gardenview Pl,Dublin
25,For Sale,3,2,"1,304","$240,000",562 Arden St,Lewis Center
26,For Sale,4,2.5,"1,909","$344,360",1635 Lewes Castle Dr,Grove City
27,For Sale,2,2,"1,441","$308,470",1627 Lewes Castle Dr,Grove City
...,...,...,...,...,...,...,...
2094,For Sale,3,3.5+,"2,280","$866,500","292 Dickenson St,","Columbus, OH 43215"
2095,For Sale,4,"4,050","9,583","$394,900","177 N James Rd,","Columbus, OH 43213"
2096,For Sale,1.79,NA,NA,"$199,900","Norton Rd Lot 3,","Columbus, OH 43228"
2097,For Sale,2,2,"1,270","$603,000","695 Civitas Ave Unit 203,","Columbus, OH 43215"


In [ ]:
for_sale_homes['']